# 国勢調査５００ｍメッシュデータの加工
- E-stat;国勢調査（５００ｍメッシュ）
- https://www.e-stat.go.jp/gis/statmap-search?page=1&type=1&toukeiCode=00200521
- 年度ごとの総人口を１つのcsvファイルにまとめる

In [ ]:
import glob
import re
import os
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt

In [ ]:
# Jupyter Notebookでノートブック上にグラフを描画する際に指定する記述
%matplotlib inline

## １）変数を設定


In [ ]:
# 都道府県
region = "Tokyo"

In [ ]:
# メッシュID
meshID = "5339"

In [ ]:
dir = "../Data/Raw"

## ２）５００ｍのシェープデータ(shp)を読み込み(data)
- geopandasライブラリを利用
- （参考）https://www.gis-py.com/entry/geopandas-master

In [ ]:
fp = dir + "/" + region + "Mesh/HXYSWH" + meshID + "/MESH0" + meshID +".shp"

In [ ]:
fp

In [ ]:
data = gpd.read_file(fp, encoding="SHIFT-JIS")

In [ ]:
# GeoDataFrameの最初の5行を取得
data.head()

## ３）"KEY_CODE"をindexにするため、"key_code"を新たに作成してを複製
- （参考）https://note.nkmk.me/python-pandas-set-index/

In [ ]:
data["key_code"] = data["KEY_CODE"]

In [ ]:
data_1 = data.set_index("key_code")

In [ ]:
data_1.head()

## ４）年度ごとの同一メッシュのtxtファイルをリスト(allFiles)に保存
- （参考）https://note.nkmk.me/python-glob-usage/

In [ ]:
allFiles = sorted(glob.glob(dir + "/" + region + "*/*/*" + meshID + "/*" + meshID + ".txt"))

In [ ]:
allFiles

## ５）pandasでリストにまとめたtxtファイルを読み込みリスト化(list_1)
- （参考）https://www.dragonarrow.work/articles/202
- DataFrameの列("KEY_CODE")をインデックス（行名）に割り当てる
- （参考）https://note.nkmk.me/python-pandas-set-index/

In [ ]:
list_1 = []

In [ ]:
for file_ in allFiles:
    df = pd.read_table(file_,
                   encoding = "shift_jis",
                   sep = ",",
                   dtype = 'object',
                   index_col=None, 
                   header=0)
    df = df.set_index("KEY_CODE")
    list_1.append(df)

In [ ]:
list_1

## ６）メッシュ(data_1)と人口(list_1)を結合(join)
- （参考）https://note.nkmk.me/python-pandas-merge-join/
- （参考）https://deepage.net/features/pandas-join.html

In [ ]:
data_2 = data_1.join(list_1)

In [ ]:
data_2.head()

In [ ]:
# 各年の総人口のカラムを抽出
data_3 = data_2.loc[:,["KEY_CODE", "T000752001","T000386001","T000387001","T000609001","T000847001","geometry"]]

## ７）カラム名を変更
- （参考）https://note.nkmk.me/python-pandas-dataframe-rename/
- pandas.DataFrameの構造とその作成方法
- （参考）https://note.nkmk.me/python-pandas-dataframe-values-columns-index/

In [ ]:
data_4 = data_3.rename(columns={"T000752001":"1995_Total","T000386001":"2000_Total","T000387001":"2005_Total","T000609001":"2010_Total","T000847001":"2015_Total"})

In [ ]:
data_4.head()

## ８）データ型を確認
- https://note.nkmk.me/python-pandas-dtype-select/

In [ ]:
data_4.dtypes

## ９）欠損値NaNを０で置換
（参考）https://note.nkmk.me/python-pandas-nan-dropna-fillna/

In [ ]:
data_5 = data_4.fillna({'1995_Total': 0, '2000_Total': 0, '2005_Total': 0, '2010_Total' : 0,  '2015_Total' : 0})

In [ ]:
data_5

## １０）データ型を整数(int)に変更
- （参考）https://note.nkmk.me/python-pandas-dataframe-values-columns-index/
- （参考）https://note.nkmk.me/python-pandas-dtype-astype/

In [ ]:
data_6= data_5.astype({'1995_Total': 'int', '2000_Total': 'int', '2005_Total': 'int', '2010_Total' : 'int',  '2015_Total' : 'int'})

In [ ]:
data_6.head()

## １１）結合したデータ(data_6)の確認
- （参考）https://note.nkmk.me/python-pandas-len-shape-size/
- （参考）https://note.nkmk.me/python-pandas-idxmax-idxmin/
- （参考）https://note.nkmk.me/python-pandas-median/

In [ ]:
# 行数や列数などを表示
data_6.info()

In [ ]:
# データ型を確認
data_6.dtypes

In [ ]:
# オブジェクトの型を取得
type(data_6)

In [ ]:
# 最大値
data_6["2000_Total"].max()

In [ ]:
# 平均
data_6["2000_Total"].mean()

In [ ]:
# 中央値
data_6["2000_Total"].median()

In [ ]:
# 最小値
data_6["2000_Total"].min()

## １２）Fionaを使用して測地系の設定
- （参考）https://www.gis-py.com/entry/geopandas-master

In [ ]:
from fiona.crs import from_epsg

In [ ]:
data_6.crs = from_epsg(2451)

In [ ]:
data_6.crs

## １３）shpファイルに書き出し


In [ ]:
dir_1 = "../Data/Working"

In [ ]:
dir_2 = dir_1 + "/" + region +"_MeshPop" + "/" + region + meshID

In [ ]:
dir_2

In [ ]:
os.mkdir(dir_2)

In [ ]:
data_6.to_file(dir_2)